In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 30

## Prediction

Let's revisit Galton's predictions of children's heights based on their parent's heights...

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )
heights

In [ ]:
# look at a scatter plot of the relationship 
heights.scatter('MidParent')

In [ ]:
def predict_child(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.25 inches.
    """
    
    close_points = heights.where('MidParent', are.between(h-0.5, h + 0.5))
    return close_points.column('Child').mean()   

In [ ]:
# predict the height for each child in the data set
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
    )

In [ ]:
# visualize the predicted heights
heights_with_predictions.scatter('MidParent')

## Association

Data on hybrid passenger cars sold in US 1997-2013:

- `vehicle`: model of the car
- `year`: year of manufacture
- `msrp`: manufacturer's suggested retail price in 2013 dollars
- `acceleration`: acceleration rate in km per hour per second
- `mpg`: fuel econonmy in miles per gallon
- `class`: the model's class.

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.sort('msrp', descending=True)

In [ ]:
# vehicles with higher mpg tend to cost less on average - surprising?
hybrid.scatter('mpg', 'msrp')

In [ ]:
# vehicles that accelerate faster tend to cost more, and have lower mpg (not as fuel efficient)
hybrid.scatter('acceleration', 'msrp')

### Correlation coefficient

In [ ]:
def r_scatter(r):
    plots.figure(figsize=(5,5))
    "Generate a scatter plot with a correlation approximately r"
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

In [ ]:
r_scatter(.3)

In [ ]:
r_scatter(0)

In [ ]:
r_scatter(-0.2)

In [ ]:
r_scatter(-.95)

In [ ]:
r_scatter(-.7)

### Calculating the correlation coefficient

To calculate the correlation coefficient r, we first convert our data to standardized units (by z-scoring out data)


In [ ]:
# Convert data to standard units
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x))/np.std(x)  

In [ ]:
hybrid

In [ ]:
hybrid2 = hybrid.select("msrp", "acceleration")

hybrid2 =  hybrid2.with_columns(
    'msrp (standard units)', standard_units(hybrid.column('msrp')),
    'acceleration (standard units)', standard_units(hybrid.column('acceleration')), 
)
hybrid2

In [ ]:
# Use of standard units does not change the point patterns
hybrid2.scatter('acceleration (standard units)', 'msrp (standard units)')

In [ ]:
# Use of standard units does not change the point patterns
hybrid2.scatter('acceleration', 'msrp')

In [ ]:
# we then calculate the product of the standardized units
hybrid2 = hybrid2.with_column(
    'product of standard units', hybrid2.column('acceleration (standard units)') * hybrid2.column('msrp (standard units)'))

hybrid2

In [ ]:
# r is the average of the products of standard units
r = np.sum(hybrid2.column('product of standard units'))/(hybrid2.num_rows -1)
r

In [ ]:
# we can create a function to calculate the correlation coefficient
def correlation(t, label_x, label_y):
    x_in_standard_units = standard_units(t.column(label_x))
    y_in_standard_units = standard_units(t.column(label_y))
    return np.sum(x_in_standard_units * y_in_standard_units)/(t.num_rows - 1)

In [ ]:
# calculate correlation between acceleration and msrp
correlation(hybrid, 'acceleration', 'msrp')

In [ ]:
# calculate correlation between mpg and msrp
correlation(hybrid, 'mpg', 'msrp')

In [ ]:
# order doesn't matter
correlation(hybrid, 'msrp', 'mpg')

## Correlation cautions

In [ ]:
# correlation only captures linear trends
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, "x", "y")

In [ ]:
# correlation is heavily influenced by outliers
anscombes = Table.read_table('anscombes.csv')
data1 = anscombes.where("dataset", "I")
data2 = anscombes.where("dataset", "II")
data3 = anscombes.where("dataset", "III")
data4 = anscombes.where("dataset", "IV")

In [ ]:
# data set 1
data1.scatter("x", "y")
plots.title("r = " + str(round(correlation(data1, "x", "y"), 3)));

In [ ]:
# data set 2
data2.scatter("x", "y")
plots.title("r = " + str(round(correlation(data2, "x", "y"), 3)));

In [ ]:
# data set 3
data3.scatter("x", "y")
plots.title("r = " + str(round(correlation(data3, "x", "y"), 3)));

In [ ]:
# data set 4
data4.scatter("x", "y")
plots.title("r = " + str(round(correlation(data4, "x", "y"), 3)));

## Linear regression

In [ ]:
# original scatter plot of the data and the correlation for the Galton data
heights.scatter('MidParent')

In [ ]:
# predictions made by taking average of children's heights in a neighborhood
heights_with_predictions.scatter('MidParent')

In [ ]:
# predictions made by the regression line
heights.scatter('MidParent', fit_line = True)

In [ ]:
# comparing prediction of the regression line and average in a neighborhood
heights_with_predictions.scatter('MidParent', fit_line = True)

## Regression in standardized units

In [ ]:
# Let's look at the relationship in standaridized units (z-score transformed units)

heights_standardized = Table().with_columns(
    "MidParent", standard_units(heights.column("MidParent")),
    "Child", standard_units(heights.column("Child"))
)

heights_standardized

In [ ]:
heights_standardized.scatter("MidParent", fit_line = True)

In [ ]:
# Correlation between children's and parent's heights
correlation(heights, "Child", "MidParent")

In [ ]:
# predictions are less than the identity line -> regression to the mean
heights_standardized.scatter("MidParent", fit_line = True)
plots.plot(make_array(-3, 3), make_array(-3, 3), color="red");

In [ ]:
# function to calculate the slope 
def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd



In [ ]:
# slope for predicting child's height
slope(heights, "MidParent", "Child")

# Q: for every additional inch a parent is taller, how much taller is the predicted child's height?

In [ ]:
# function to calculate the intercept
def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean


In [ ]:
# intercept for predicting child's height
intercept(heights, "MidParent", "Child")

#Q: How a parents that are 0" tall, how tall is the predicted height of their child? 

### Regression equation for Galton data

$\hat{y} = 0.638 \cdot x + 22.589$



In [ ]:
# How tall would be predict a child to be if their parents were 70 inches?
x = 72
0.638 * x + 22.589